In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://ews-tools.de"

response = requests.get(url)

soup = BeautifulSoup(response.text, 'html5lib')

links = soup.find_all('a', href=True)
# print(soup.prettify())

# for link in soup.find_all('a'):
#     print(link.get('href'))

career_keywoards = {"career", "karriere", "jobs"}

apply_keywoards = {"bewerben", "bewerber", "apply", "bewerbung"}

links = soup.find_all('a', href=True)
links = [(link_obj, link_obj.get("href")) for link_obj in links]

job_links = []

for _, link_name in links:
    if link_name is not None:
        for keywoard in career_keywoards:
            if keywoard in link_name.lower():
                job_links.append(url + link_name)

job_link = list(set(job_links))[1]


job_soup = BeautifulSoup(requests.get(job_link).text, 'html5lib')
links = job_soup.find_all('a', href=True)
links = [(link_obj, link_obj.get("href")) for link_obj in links]

apply_links = []
for _, link_name in links:
    if link_name is not None:
        for keywoard in apply_keywoards:
            if keywoard in link_name.lower():
                apply_links.append(link_name)

apply_links


In [13]:
from pypdf import PdfReader
import os

pdfs = os.listdir("/home/tom/Downloads/Kontoauszuege-20250101T185341Z-001/Kontoauszuege")
pdfs = [pdf for pdf in pdfs if pdf.endswith(".pdf")]
pdf_pages = []
for pdf in pdfs:
    reader = PdfReader(f"/home/tom/Downloads/Kontoauszuege-20250101T185341Z-001/Kontoauszuege/{pdf}")
    pdf_pages.append([reader.pages[i] for i in range(reader.get_num_pages()-1)])


In [44]:
len(pdf_pages)

12

In [46]:
import re

def extract_monthly_transaction_strings(pdf_pages):
    # Regex pattern to match transactions with their date and details
    pattern = r"(\d{2}\.\d{2}\.\s+\d{2}\.\d{2}\.)\s+((?!Abschluss|neuer Kontostand).+?)(?=\n\d{2}\.\d{2}\.|Übertrag|$)"
    monthly_transactions = []
    for page in pdf_pages:
        transactions = re.findall(pattern, page.extract_text(), re.DOTALL)
        for transaction_date, transaction_details in transactions:
            # Clean up the details: remove excessive indentation
            formatted_details = transaction_date[:6].strip() + "\n" + "\n".join(line.strip() for line in transaction_details.splitlines())
            monthly_transactions.append(formatted_details)
    return monthly_transactions

def get_month_date_from_pdf(pdf_pages):
    date_pattern = r"neuer\s+Kontostand\s+vom\s+(\d{2}\.\d{2}\.\d{4})"
    month_date = re.findall(date_pattern, pdf_pages[-1].extract_text())[0]
    return month_date[3:]

yearly_transactions = {}

for pdf_monthly in pdf_pages:
    month_date = get_month_date_from_pdf(pdf_monthly)
    yearly_transactions[month_date] = extract_monthly_transaction_strings(pdf_monthly)

yearly_transactions = dict(sorted(yearly_transactions.items()))
transaction_types = []
for month in yearly_transactions.keys():
#    print(f"Datum: {transaction[0]}")
#    if "ÜBERWEISUNG" in transaction:
    for transaction in yearly_transactions[month]:
        if "PayPal" in transaction: 
            print(transaction, "\n")
            transaction_types.append(transaction.split("\n")[1][:20])

print("\n\n")
transaction_types = set(transaction_types)
for transaction in transaction_types:
    print(transaction)


03.01.
Basislastschrift PN:931                                              43,46 S
PayPal Europe S.a.r.l. et Cie S.C.A
1031627271692/PP.7133.PP/. M 5 Gastronomie GmbH, Ihr E
inkauf bei M 5 Gastronomie GmbH EREF: 1031627271692 MR
EF: 5LB22253SMGR4 CRED: LU96ZZZ0000000000000000058 

15.01.
EURO-GUTSCHRIFT PN:931                                                        100,00 H
PayPal Europe S.a.r.l. et Cie S.C.A
PP.7133.PP/ABBUCHUNG VOM PAYPAL-KONTO EREF: YYW1031874
484600 

16.01.
Basislastschrift PN:931                                              56,00 S
PayPal Europe S.a.r.l. et Cie S.C.A
1031874474523/PP.7133.PP/. , Ihr Einkauf bei EREF: 103
1874474523 MREF: 5LB22253SMGR4 CRED: LU96ZZZ0000000000
000000058 

22.01.
EURO-GUTSCHRIFT PN:931                                                        179,02 H
PayPal Europe S.a.r.l. et Cie S.C.A
PP.7133.PP/ABBUCHUNG VOM PAYPAL-KONTO EREF: YYW1032061
328045 

26.01.
Basislastschrift PN:931                                               1,05 S
Pay

In [53]:
try:
    orderer = re.search(
    r"(.*?)(?=, Ihr Einkauf bei)",
    "PayPal Europe S.a.r.l. et Cie S.C.APP.7133.PP/ABBUCHUNG VOM PAYPAL-KONTO EREF: YYW1038642256739 ",
    re.IGNORECASE,
    ).group(0)
except AttributeError:
    orderer = "account_holder" 
orderer

'account_holder'

In [29]:
transaction_lines = "060-319/24-00031 TAN: Auftrag nicht TAN-pflichtig, da"

transaction_purpose = re.match(r'^(.*?)(?=\s*(SecureGo|TAN))', transaction_lines)
transaction_purpose.group(1)

'060-319/24-00031'